In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

def floyd_warshall(graph):
    INF = 999
    n = len(graph)

    dist = [[INF for _ in range(n)] for _ in range(n)]

    for i in range(n):
        for j in range(n):
            if i == j:
                dist[i][j] = 0
            elif graph[i][j] > 0:
                dist[i][j] = graph[i][j]

    pred = [[-1 if i != j and graph[i][j] == 0 else i for j in range(n)] for i in range(n)]

    for k in range(n):
        for i in range(n):
            for j in range(n):
                if dist[i][j] > dist[i][k] + dist[k][j]:
                    dist[i][j] = dist[i][k] + dist[k][j]
                    pred[i][j] = pred[k][j]
    return dist, pred

def print_matrix(matrix, labels):
    print(f"\nMatrix of shortest distances between each pair of nodes:")
    n = len(matrix)
    for i in range(n):
        for j in range(n):
            if matrix[i][j] == 999:
                print("INF", end=" ")
            else:
                print(f"{matrix[i][j]:3d}", end=" ")
        print()

def reconstruct_path(i, j, pred):
    path = []
    if pred[i][j] == -1:
        return path
    while i != j:
        path.append(j)
        j = pred[i][j]
    path.append(i)
    path.reverse()
    return path

def print_routes(origin, pred, dist, labels):
    n = len(pred)
    print(f"\nRoutes of shortest paths from {labels[origin]} to other nodes:")
    for j in range(n):
        if origin != j and pred[origin][j] != -1:
            path = reconstruct_path(origin, j, pred)
            route = ' -> '.join([labels[v] for v in path])
            length = dist[origin][j]
            print(f"From {labels[origin]} to {labels[j]}: {route} ({length})")

def plot_digraph(graph, labels, title):
    G = nx.DiGraph()

    edge_labels = {}

    for i in range(len(graph)):
        for j in range(len(graph[i])):
            if graph[i][j] > 0:
                G.add_edge(i, j, weight=graph[i][j])
                edge_labels[(i, j)] = graph[i][j]

    pos = nx.circular_layout(G)

    plt.figure(figsize=(6, 6))
    nx.draw(G, pos, with_labels=True, labels={i: labels[i] for i in range(len(labels))},
            node_color='lightblue', node_size=800, font_size=10, font_color='black',
            edge_color='black', linewidths=1, edgecolors='black', arrows=True, arrowsize=20)

    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_color='black', rotate=False, label_pos=0.4)
    plt.title(title, fontsize=14, y=1.02)
    plt.show()

labels = ['City A', 'City B', 'City C', 'City D', 'City E', 'City F', 'City G', 'City H']

adj_matrix = [
    [0, 13,  0,  0,  0,  0,  0,  0],
    [0,  0,  0, 16,  0,  0,  8,  0],
    [11, 0,  0,  0,  5,  0,  0, 14],
    [0, 16,  0,  0,  0,  7,  0,  0],
    [0,  0,  5,  0,  0,  9,  0,  0],
    [0,  0,  0,  0,  9,  0, 12,  0],
    [0,  0, 18,  0,  0,  0,  0, 10],
    [0,  0,  0,  0,  0,  0,  0,  0],
]

shortest_paths, predecessors = floyd_warshall(adj_matrix)

print("Available cities:")
for i, city in enumerate(labels, 1):
    print(f"{i}-{city}")

origin = int(input(f"\nChoose a starting city (1 to 8): ")) - 1

print_matrix(shortest_paths, labels)
print_routes(origin, predecessors, shortest_paths, labels)

plot_digraph(adj_matrix, labels, "City Network")
